# Prepare for each projects

In [1]:
!pip install fabric

Looking in indexes: https://pypi.org/simple, http://acoe.connect.te.com
  Using cached fabric-2.5.0-py2.py3-none-any.whl (51 kB)
  Using cached paramiko-2.7.2-py2.py3-none-any.whl (206 kB)
  Using cached invoke-1.4.1-py3-none-any.whl (210 kB)
  Using cached PyNaCl-1.4.0-cp35-abi3-manylinux1_x86_64.whl (961 kB)
  Using cached bcrypt-3.2.0-cp36-abi3-manylinux2010_x86_64.whl (63 kB)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, http://acoe.connect.te.com
  Using cached python_dotenv-0.15.0-py2.py3-none-any.whl (18 kB)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
from fabric import Connection, Config

In [4]:
from dotenv import load_dotenv
import os

In [90]:
load_dotenv()

True

In [6]:
pwd = os.getenv('PI_PSWD')
pi1 = os.getenv('PI_IP1')

In [7]:
config = Config({'user': 'pi', 'connect_kwargs': {'password': pwd}})

In [8]:
cxn = Connection(pi1, config = config)

In [9]:
cxn.run('whoami')

pi


<Result cmd='whoami' exited=0>

In [10]:
cxn.sudo('pip3 install --upgrade pip')

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Attempting uninstall: pip
    Found existing installation: pip 20.2.3
    Uninstalling pip-20.2.3:
      Successfully uninstalled pip-20.2.3


<Result cmd="sudo -S -p '[sudo] password: ' pip3 install --upgrade pip" exited=0>

In [15]:
# cxn.sudo('pip install virtualenv')

# Redis Server (Docker)

In [24]:
import requests
from redis import Redis
from rq import Queue
from rq_scheduler import Scheduler

from datetime import datetime
import pytz

In [37]:
# cxn.run('redis-server --version')

In [53]:
host = 'redis_local' #os.getenv('REDIS_HOST')
port = os.getenv('REDIS_PORT')

In [59]:
r = Redis(host, port)

In [55]:
r

Redis<ConnectionPool<Connection<host=redis_local,port=6379,db=0>>>

In [56]:
r.set('example', '1928')

True

In [60]:
r.get('example')

b'1928'

In [57]:
scheduler = Scheduler(connection = r)

In [58]:
scheduler.count()

0

# Redis Raspberry pi

In [91]:
host = os.getenv('REDIS_HOST')
port = os.getenv('REDIS_PORT')
redis_password = os.getenv('REDIS_PASSWORD_PI')

In [64]:
# cxn.sudo('pip3 install redis')

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple


<Result cmd="sudo -S -p '[sudo] password: ' pip3 install redis" exited=0>

In [70]:
# cxn.sudo('apt-get install redis -y')

In [111]:
# cxn.sudo('kill -9 32008  ') # Not working
#cxn.sudo('service redis-server stop')

<Result cmd="sudo -S -p '[sudo] password: ' kill -9 32008  " exited=0>

In [113]:
# cxn.sudo('ps aux | grep redis')

In [114]:
# cxn.sudo(f'service redis-server restart ./redis.conf') #service redis-server restart
# cxn.sudo('service redis-server restart --protected-mode no')
cxn.sudo('redis-server --protected-mode no')

32333:C 07 Dec 2020 21:29:08.839 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
32333:C 07 Dec 2020 21:29:08.839 # Redis version=5.0.3, bits=32, commit=00000000, modified=0, pid=32333, just started
32333:C 07 Dec 2020 21:29:08.840 # Configuration loaded
32333:M 07 Dec 2020 21:29:08.850 * Increased maximum number of open files to 10032 (it was originally set to 1024).
32333:M 07 Dec 2020 21:29:08.851 # Warning: 32 bit instance detected but no memory limit set. Setting 3 GB maxmemory limit with 'noeviction' policy now.
32333:M 07 Dec 2020 21:29:08.855 * Running mode=standalone, port=6379.
32333:M 07 Dec 2020 21:29:08.856 # Server initialized
32333:M 07 Dec 2020 21:29:08.856 # WARNING overcommit_memory is set to 0! Background save may fail under low memory condition. To fix this issue add 'vm.overcommit_memory = 1' to /etc/sysctl.conf and then reboot or run the command 'sysctl vm.overcommit_memory=1' for this to take effect.
32333:M 07 Dec 2020 21:29:08.857 * Ready to accept connections


KeyboardInterrupt: 

In [117]:
r = Redis(host, port)

In [122]:
r.set('foo', 'bar1', px = 100_000)

True

In [129]:
r.get('foo')

b'bar1'

In [130]:
r.ttl('foo')

57

In [133]:
r.sadd('pythonlist', '1', '2', '3', '4')

4

In [134]:
r.sadd('powershelllist', '2', '4')

2

In [135]:
r.sinter('pythonlist', 'powershelllist')

{b'2', b'4'}

In [136]:
r.sunion('pythonlist', 'powershelllist')

{b'1', b'2', b'3', b'4'}

In [140]:
r.scard('pythonlist')

0

In [141]:
r.expire('pythonlist', 1)

False

In [143]:
r.expire('powershelllist',1)

False

# Saving pandas to Redis

In [147]:
import pyarrow as pa
import pandas as pd

In [144]:
def storeInRedis(alias, df):
    df_compressed = pa.serialize(df).to_buffer().to_pybytes()
    res = r.set(alias,df_compressed)
    if res == True:
        print(f'{alias} cached')

def loadFromRedis(alias):
    data = r.get(alias)
    try:
        return pa.deserialize(data)
    except:
        print("No data")

In [148]:
dd = {'ID': ['H576','H577','H578','H600', 'H700'],
      'CD': ['AAAAAAA', 'BBBBB', 'CCCCCC','DDDDDD', 'EEEEEEE']}
df = pd.DataFrame(dd)

In [149]:
storeInRedis('pandas', df)

pandas cached


In [150]:
loadFromRedis('pandas')

,ID,CD
0,H576,AAAAAAA
1,H577,BBBBB
2,H578,CCCCCC
3,H600,DDDDDD
4,H700,EEEEEEE


In [151]:
r.expire('pandas', 10)

True